In [1]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl

In [2]:
# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

In [3]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:02<00:00, 4555665.69it/s]


Extracting c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw\train-images-idx3-ubyte.gz to c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28610225.28it/s]


Extracting c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw\train-labels-idx1-ubyte.gz to c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 7470235.14it/s]


Extracting c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw\t10k-images-idx3-ubyte.gz to c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]


Extracting c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw\t10k-labels-idx1-ubyte.gz to c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\MNIST\raw



In [4]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\nilsm\workspace\MakeAIWork2\env\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
Missing logger folder: c:\Users\nilsm\workspace\MakeAIWork2\practica\07_ml_frameworks\lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
-------------------------------

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 5.3302e+36, -7.9008e+35, -1.8957e+36],
        [-9.1644e+35,  1.8408e+36, -7.6807e+36],
        [-8.4109e-02, -6.5780e-02,  3.3199e-03],
        [-8.4109e-02, -6.5780e-02,  3.3199e-03]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
